In [6]:
import joblib
import pandas as pd
from python_files.data_preprocessor import preprocess_test
import requests

In [5]:
model = joblib.load('models\lgbm_classifier.pkl')
scaler = joblib.load('models\lgbm_scaler.pkl')
encoder = joblib.load('models\lgbm_encoder.pkl')

In [7]:
url = "http://127.0.0.1:8000/predict"

data = {
  "Transaction_Id": "6561289625",
  "Date_Time": "2021-05-13 21:26:26",
  "Account_Creation_Date": "2021-05-12 21:26:26",
  "Phone_Number": "+92 327-5724582",
  "CNIC": "15352-7462017-7",
  "Name": "Maira Butt",
  "Type": "PAYMENT",
  "Amount": 9839.64,
  "ID_Source": "C1231006815",
  "Old_Balance": 170136.0,
  "New_Balance": 160296.36,
  "Source_State": "Gilgit-Baltistan",
  "Source_City": "Gilgit",
  "Device_Name": "Google Pixel 6",
  "IMEI": "831780764842",
  "KYC_Status": "On-Hold",
  "Service_Charges": 19.67928,
  "Channel": "Credit Card",
  "Remarks": "",
  "ID_Dest": "M1979787155",
  "Dest_State": "Gilgit-Baltistan",
  "Dest_City": "Gilgit"
}


In [8]:
data_fraud = {'Transaction_Id': '5720078885', 'Date_Time': '2021-04-05 18:32:15', "Account_Creation_Date": "2021-04-05 18:30:26",'Phone_Number': '+92 305-3516186', 'CNIC': '63643-6029379-9', 'Name': 'Bilal Qureshi', 'Type': 'TRANSFER', 'Amount': 837192.23, 'ID_Source': 'C653262851', 'Old_Balance': 837192.23, 'New_Balance': 0.0, 'Source_State': 'Azad Kashmir', 'Source_City': 'Rawalakot', 'Device_Name': 'OnePlus 9', 'IMEI': '468898808901590', 'KYC_Status': 'Validated', 'Service_Charges': 1674.38446, 'Channel': 'Bank Transfer', 'Remarks': '', 'ID_Dest': 'C690345380', 'Dest_State': 'Gilgit-Baltistan', 'Dest_City': 'Skardu'}

In [16]:
df = pd.DataFrame([data])

In [17]:
processed_data = preprocess_test(df,scaler,encoder)

In [18]:
prediction = model.predict_proba(processed_data)

In [19]:
prediction

array([[9.99996939e-01, 3.06064904e-06]])

In [4]:
response = requests.post(url, json=data_fraud)
print(response.json())

{'prediction Model LGBM': [1], 'Prediction Business Rules': {'BusinessRule1_rule1_n_transactions_m_minutes': False, 'BusinessRule2_rule2_high_risk_cities': False, 'BusinessRule3_rule3_high_risk_region_and_time': False, 'BusinessRule4_rule4_new_account_with_large_transaction': False}, 'Prediction Statiscal Rules': {'StatisticalRule5_rule5_large_transaction': True, 'StatisticalRule6_ule6_high_frequency_same_source': False, 'StatisticalRule7_rule7_unusual_dest_accounts': False, 'StatisticalRule8_rule8_time_based_spike': False}}


In [36]:
print(response.json()['Prediction Business Rules'])

{'BusinessRule1_rule1_n_transactions_m_minutes': False, 'BusinessRule2_rule2_high_risk_cities': False, 'BusinessRule3_rule3_high_risk_region_and_time': False, 'BusinessRule4_rule4_new_account_with_large_transaction': False}


In [ ]:
response = requests.post(url, json=data)
print(response.json()['Flag Results'])
response = requests.post(url, json=data_fraud)
print(response.json())
response = requests.post(url, json=data_fraud)
print(response.json())

{'flagged_by_rule': {'Rule1:n_transactions_m_minutes': [], 'Rule2:high_risk_cities': [], 'Rule3:high_risk_region_and_time': [], 'Rule4:ew_account_with_large_transaction': [], 'Rule5:arge_transaction': [], 'Rule6:igh_frequency_same_source': [], 'Rule7:unusual_dest_accounts': [], 'Rule8:time_based_spike': []}, 'combined_flagged_transactions': []}


In [47]:
df['Transaction_Id'][0]

'6561289625'

In [55]:
from Rules__ import FraudRulesEngine

In [56]:
df = pd.DataFrame([data])

engine = FraudRulesEngine(df)

transaction_id=df['Transaction_Id'][0]

result = engine.apply_all_rules(transaction_id=transaction_id)

In [57]:
result

{'flagged_by_rule': {'Rule1:n_transactions_m_minutes': [],
  'Rule2:high_risk_cities': [],
  'Rule3:high_risk_region_and_time': [],
  'Rule4:ew_account_with_large_transaction': [],
  'Rule5:arge_transaction': [],
  'Rule6:igh_frequency_same_source': [],
  'Rule7:unusual_dest_accounts': [],
  'Rule8:time_based_spike': []},
 'combined_flagged_transactions': []}

In [53]:
if result:
    print(f"\nFlag Status for Transaction {transaction_id}:\n")

    for rule_name, flagged_ids in result['flagged_by_rule'].items():
        status = "TRUE" if transaction_id in flagged_ids else "FALSE"
        print(f"{rule_name}: {status}")

    print("\nCombined Flagged Transactions:")
    print(result['combined_flagged_transactions'])

else:
    print("Transaction not found or error occurred.")


Flag Status for Transaction 6561289625:

Rule1:n_transactions_m_minutes: FALSE
Rule2:high_risk_cities: FALSE
Rule3:high_risk_region_and_time: FALSE
Rule4:ew_account_with_large_transaction: FALSE
Rule5:arge_transaction: FALSE
Rule6:igh_frequency_same_source: FALSE
Rule7:unusual_dest_accounts: FALSE
Rule8:time_based_spike: FALSE

Combined Flagged Transactions:
[]


In [11]:
from rules import apply_business_rules

In [12]:
df = pd.DataFrame([data])

In [16]:
from data_preprocessor import preprocess_test

In [13]:
res = preprocess_test(df,scaler,encoder)

In [14]:
res

,Type,Amount,ID_Source,Old_Balance,New_Balance,Source_State,Source_City,Device_Name,KYC_Status,Service_Charges,...,Day_DT,Hour_DT,Weekday_DT,Day_of_year_DT,Year_Acr_DT,Month_Acr_DT,Day_Acr_DT,Hour_Acr_DT,Weekday_Acr_DT,Day_of_year_Acr_DT
0,4.0,0.000106,556958.0,0.003883,0.003669,3.0,14.0,1.0,1.0,0.000106,...,13,21,3,133,2021,5,12,21,2,132


In [15]:
res['Date_Time'] = '2025-03-15 17:12:12'

In [16]:
results = apply_business_rules(res)

In [17]:
results

,Type,Amount,ID_Source,Old_Balance,New_Balance,Source_State,Source_City,Device_Name,KYC_Status,Service_Charges,...,time_diff,new_session,session_id,session_txn_count,median_count,flag_heavy_session,flag_rapid_txns,flag_high_risk_city,flag_risky_time_region,flag_new_acc_large_amt
0,4.0,0.000106,556958.0,0.003883,0.003669,3.0,14.0,1.0,1.0,0.000106,...,NaN,True,1,1,1.0,False,0,False,False,False


In [20]:
results['New Account Large Amount'][0]

KeyError: 'New Account Large Amount'

In [3]:
# test.py

import pandas as pd
from datetime import datetime

# Import the classes from Rules.py
from Rules_ import FraudRulesEngine



In [4]:
df = pd.DataFrame([data])

In [5]:
df

,Transaction_Id,Date_Time,Account_Creation_Date,Phone_Number,CNIC,Name,Type,Amount,ID_Source,Old_Balance,...,Source_City,Device_Name,IMEI,KYC_Status,Service_Charges,Channel,Remarks,ID_Dest,Dest_State,Dest_City
0,6561289625,2021-05-13 21:26:26,2021-05-12 21:26:26,+92 327-5724582,15352-7462017-7,Maira Butt,PAYMENT,9839.64,C1231006815,170136.0,...,Gilgit,Google Pixel 6,831780764842,On-Hold,19.67928,Credit Card,,M1979787155,Gilgit-Baltistan,Gilgit


In [29]:
df=pd.read_csv('test_data.csv')

In [30]:
txn = dict(df.iloc[0])

In [31]:
# Initialize the FraudRulesEngine with the test DataFrame
engine = FraudRulesEngine(txn)


In [32]:

# Apply all fraud detection rules
result = engine.apply_all_rules()


AttributeError: 'dict' object has no attribute 'iterrows'

In [9]:
result

{'flagged_by_rule': {'n_transactions_m_minutes': [],
  'high_risk_cities': [],
  'high_risk_region_and_time': [],
  'new_account_with_large_transaction': [],
  'large_transaction': [],
  'high_frequency_same_source': [],
  'unusual_dest_accounts': [],
  'time_based_spike': []},
 'combined_flagged_transactions': []}

In [13]:
# Print the flagged transactions for each rule
print("Flagged Transactions by Rule:")
for rule, txns in result['flagged_by_rule'].items():
    print(f"{rule}: {len(txns)}")

Flagged Transactions by Rule:
n_transactions_m_minutes: 0
high_risk_cities: 0
high_risk_region_and_time: 0
new_account_with_large_transaction: 0
large_transaction: 0
high_frequency_same_source: 0
unusual_dest_accounts: 0
time_based_spike: 0


In [16]:


# Print the flagged transactions for each rule
print("Flagged Transactions by Rule:")
for rule, txns in result['flagged_by_rule'].items():
    print(f"{rule}: {len(txns)}")


Flagged Transactions by Rule:
n_transactions_m_minutes: 0
high_risk_cities: 0
high_risk_region_and_time: 0
new_account_with_large_transaction: 0
large_transaction: 0
high_frequency_same_source: 0
unusual_dest_accounts: 0
time_based_spike: 0


In [33]:
import pandas as pd
from datetime import datetime

# Import the classes from Rules.py
from Rules__ import FraudRulesEngine



In [35]:


data = {
    'Transaction_Id': '262874940',
    'Date_Time': '2025-11-22 03:51:50',
    'Account_Creation_Date': '2025-11-20 07:51:50',
    'Phone_Number': '+92 316-5932748',
    'CNIC': '40734-8820668-3',
    'Name': 'Fariha Mirza',
    'Type': 'PAYMENT',
    'Amount': 1740100.97,
    'ID_Source': 'C1715563227',
    'Old_Balance': 9987000.00,
    'New_Balance': 0.00,
    'Device_Name': 'Xiaomi Mi 11',
    'IMEI': '601635731712294',
    'KYC_Status': 'On-Hold',
    'Service_Charges': 34.80394,
    'Channel': 'Mobile Payments',
    'Remarks': 'Utility bill payment',
    'ID_Dest': 'M1855643329',
    'Dest_State': 'Balochistan',
    'Dest_City': 'Dera Bugti',
    'Source_City': 'Dera Bugti',
    'Is_Fraud': '0',
}


# Create DataFrame from the test data
df = pd.DataFrame([data])

In [36]:
# Initialize the FraudRulesEngine with the test DataFrame
engine = FraudRulesEngine(df)

In [37]:
# Specify the Transaction ID you want to test
transaction_id = '262874940'


result = engine.apply_all_rules(transaction_id=transaction_id)


In [38]:
if result:
    print(f"\nFlag Status for Transaction {transaction_id}:\n")

    for rule_name, flagged_ids in result['flagged_by_rule'].items():
        status = "TRUE" if transaction_id in flagged_ids else "FALSE"
        print(f"{rule_name}: {status}")

    print("\nCombined Flagged Transactions:")
    print(result['combined_flagged_transactions'])

else:
    print("Transaction not found or error occurred.")



Flag Status for Transaction 262874940:

Rule1:n_transactions_m_minutes: FALSE
Rule2:high_risk_cities: TRUE
Rule3:high_risk_region_and_time: TRUE
Rule4:ew_account_with_large_transaction: TRUE
Rule5:arge_transaction: TRUE
Rule6:igh_frequency_same_source: FALSE
Rule7:unusual_dest_accounts: FALSE
Rule8:time_based_spike: FALSE

Combined Flagged Transactions:
['262874940']


In [ ]:

# Convert 'Date_Time' to datetime
#df['Date_Time'] = pd.to_datetime(df['Date_Time'])






In [60]:
import pandas as pd
from Rules import BusinessRules, StatisticalRules

In [61]:
test_df = pd.read_csv('test_data.csv')

In [62]:
data = test_df.iloc[0]

In [63]:
# Convert to DataFrame
df = pd.DataFrame([data])

In [64]:
# Set Transaction ID
transaction_id = '262874940'

print(df)

# Business Rules check
business = BusinessRules(df)
business_results = business.apply_rules(transaction_id)

   Transaction_Id            Date_Time     Phone_Number             CNIC  \
0      5471606328  2019-08-04 18:04:00  +92 310-4781920  54845-3037526-6   

          Name      Type     Amount    ID_Source  Old_Balance  New_Balance  \
0  Rida Hashmi  CASH_OUT  157548.74  C1622644134         41.0          0.0   

   ...             IMEI     KYC_Status Service_Charges          Channel  \
0  ...  606248863727928  Under Process       315.09748  Mobile Payments   

                  Remarks      ID_Dest   Dest_State Dest_City  \
0  Online gaming purchase  C1410204448  Balochistan    Chaman   

  Account_Creation_Date Is_Fraud  
0   2017-12-31 22:04:23        0  

[1 rows x 23 columns]


In [65]:
business_results

{'BusinessRule1_rule1_n_transactions_m_minutes': False,
 'BusinessRule2_rule2_high_risk_cities': False,
 'BusinessRule3_rule3_high_risk_region_and_time': False,
 'BusinessRule4_rule4_new_account_with_large_transaction': False}

In [67]:
print("Business Rules Results:")
for rule, result in business_results.items():
    print(f"{rule}: {'Flagged' if result else 'Not Flagged'}")

Business Rules Results:
BusinessRule1_rule1_n_transactions_m_minutes: Not Flagged
BusinessRule2_rule2_high_risk_cities: Not Flagged
BusinessRule3_rule3_high_risk_region_and_time: Not Flagged
BusinessRule4_rule4_new_account_with_large_transaction: Not Flagged


In [68]:
# Statistical Rules check
statistical = StatisticalRules(df)
statistical_results = statistical.apply_rules(transaction_id)

In [69]:
statistical_results

{'StatisticalRule5_rule5_large_transaction': False,
 'StatisticalRule6_ule6_high_frequency_same_source': False,
 'StatisticalRule7_rule7_unusual_dest_accounts': False,
 'StatisticalRule8_rule8_time_based_spike': False}

In [70]:
print("\nStatistical Rules Results:")
for rule, result in statistical_results.items():
    print(f"{rule}: {'Flagged' if result else 'Not Flagged'}")


Statistical Rules Results:
StatisticalRule5_rule5_large_transaction: Not Flagged
StatisticalRule6_ule6_high_frequency_same_source: Not Flagged
StatisticalRule7_rule7_unusual_dest_accounts: Not Flagged
StatisticalRule8_rule8_time_based_spike: Not Flagged
